In [28]:
from ftplib import FTP
import requests
import json
import requests
import pandas as pd
import os
import zipfile
import folium

In [29]:
def get_dados_cnefe(url):

    response = requests.get(url)

    if response.status_code == 200:

        data = response.json()

        formatted_data = json.dumps(data, indent=4)
        print(formatted_data)

        # download dos arquivos do CNEFE 
        for obj in data:
            file_url = obj['url']
            file_name = file_url.split('/')[-1]
            print(file_url, file_name)
            response = requests.get(file_url)
            if response.status_code == 200:
                with open(file_name, 'wb') as file:
                    file.write(response.content)
                print(f"Aquivo {file_name} baixado com sucesso")
            else:
                print(f"Falha ao baixar arquivo {file_name}")
    else:
        print("Falha ao obter dados do CNEFE")

In [30]:
def extract_zip():

    diretorio = "."

    # pegar todos os arquivos zip no diretorio
    zip_files = [file for file in os.listdir(diretorio) if file.endswith(".zip")]

    for zip_file in zip_files:
        zip_path = os.path.join(diretorio, zip_file)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(diretorio)
        print(f"Dados extraidos do arquivo {diretorio} para {zip_file}")

In [31]:
def concat_files():
    directory = "."  # diretório onde estão os arquivos	
    output_file = "merged.csv"  # nome do arquivo de saída

    # pegar todos os arquivos .csv no diretório
    csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

    # incializar um DataFrame vazio
    merged_df = pd.DataFrame()

    # iterar sobre cada arquivo .csv e concatenar ao DataFrame
    for csv_file in csv_files:
        csv_path = os.path.join(directory, csv_file)
        df = pd.read_csv(csv_path)
        merged_df = pd.concat([merged_df, df])

    # escrever o DataFrame concatenado em um arquivo .csv
    merged_df.to_csv(output_file, index=False)

    print(f"Todos os arquivos .csv no diretório {directory} foram concatenados em {output_file}")

In [32]:
url = "https://servicodados.ibge.gov.br/api/v1/downloads/estatisticas?caminho=Cadastro_Nacional_de_Enderecos_para_Fins_Estatisticos/Censo_Demografico_2022/Arquivos_CNEFE/UF&nivel=1"

# get_dados_cnefe(url)
# extract_zip()
# concat_files()

#### EDA

In [33]:
df = pd.read_csv("merged.csv", sep=";", encoding="latin1")

C:\Users\suelio\AppData\Local\Temp\ipykernel_5892\3073324765.py:1: DtypeWarning: Columns (18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged.csv", sep=";", encoding="latin1")


In [34]:
display(df.head())
display(df.columns)

,COD_UNICO_ENDERECO,COD_UF,COD_MUNICIPIO,COD_DISTRITO,COD_SUBDISTRITO,COD_SETOR,NUM_QUADRA,NUM_FACE,CEP,DSC_LOCALIDADE,...,VAL_COMP_ELEM5,LATITUDE,LONGITUDE,NV_GEO_COORD,COD_ESPECIE,DSC_ESTABELECIMENTO,COD_INDICADOR_ESTAB_ENDERECO,COD_INDICADOR_CONST_ENDERECO,COD_INDICADOR_FINALIDADE_CONST,COD_TIPO_ESPECI
0,186635229,25,2511400,251140005,25114000500,251140005000034P,6,2,58187000,BAIRRO CENECISTA,...,NaN,-6.510267,-36.360471,1,1,NaN,NaN,NaN,NaN,103.0
1,17796072,25,2511400,251140005,25114000500,251140005000034P,6,2,58187000,BAIRRO CENECISTA,...,NaN,-6.509939,-36.360444,1,1,NaN,NaN,NaN,NaN,101.0
2,17796073,25,2511400,251140005,25114000500,251140005000034P,6,2,58187000,BAIRRO CENECISTA,...,NaN,-6.509986,-36.360435,1,1,NaN,NaN,NaN,NaN,101.0
3,17796073,25,2511400,251140005,25114000500,251140005000034P,6,2,58187000,BAIRRO CENECISTA,...,NaN,-6.509986,-36.360435,1,6,LUCINHA CONSULTORA,1.0,NaN,NaN,NaN
4,17796074,25,2511400,251140005,25114000500,251140005000034P,6,2,58187000,BAIRRO CENECISTA,...,NaN,-6.510057,-36.360537,1,1,NaN,NaN,NaN,NaN,101.0


Index(['COD_UNICO_ENDERECO', 'COD_UF', 'COD_MUNICIPIO', 'COD_DISTRITO',
       'COD_SUBDISTRITO', 'COD_SETOR', 'NUM_QUADRA', 'NUM_FACE', 'CEP',
       'DSC_LOCALIDADE', 'NOM_TIPO_SEGLOGR', 'NOM_TITULO_SEGLOGR',
       'NOM_SEGLOGR', 'NUM_ENDERECO', 'DSC_MODIFICADOR', 'NOM_COMP_ELEM1',
       'VAL_COMP_ELEM1', 'NOM_COMP_ELEM2', 'VAL_COMP_ELEM2', 'NOM_COMP_ELEM3',
       'VAL_COMP_ELEM3', 'NOM_COMP_ELEM4', 'VAL_COMP_ELEM4', 'NOM_COMP_ELEM5',
       'VAL_COMP_ELEM5', 'LATITUDE', 'LONGITUDE', 'NV_GEO_COORD',
       'COD_ESPECIE', 'DSC_ESTABELECIMENTO', 'COD_INDICADOR_ESTAB_ENDERECO',
       'COD_INDICADOR_CONST_ENDERECO', 'COD_INDICADOR_FINALIDADE_CONST',
       'COD_TIPO_ESPECI'],
      dtype='object')

In [35]:
def exibir_mapa(df):
    df_subset = df.head(15)

    # criar um mapa
    map_cnefe = folium.Map(location=[df_subset['LATITUDE'].mean(), df_subset['LONGITUDE'].mean()], zoom_start=5)

    # adicionar marcadores ao mapa
    for index, row in df_subset.iterrows():
        # concatenando os dados para exibir no popup
        concatenated_data = f"""
            Endereço: {row['NOM_TIPO_SEGLOGR'] + ', ' + row['NOM_TITULO_SEGLOGR']}
            Bairro: {row['DSC_LOCALIDADE']}
        """ 
        folium.Marker(location=[row['LATITUDE'], row['LONGITUDE']], popup=concatenated_data).add_to(map_cnefe)


    # exibir o mapa
    display(map_cnefe)

In [36]:
exibir_mapa(df)